In [1]:
import pickle as pkl
import json
from os import listdir
import numpy as np
from os.path import isfile, join

from evaluate import compute_all_metrics

In [2]:
import torch
torch. __version__ 

'1.9.0a0+c3d40fd'

In [2]:
pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 26.7 MB 14 kB/s s eta 0:00:01 |████████████████▉               | 14.0 MB 11.3 MB/s eta 0:00:02�████████▍           | 17.0 MB 11.3 MB/s eta 0:00:01��█████████████████▋        | 19.7 MB 11.3 MB/s eta 0:00:01��████████████████████████████▎  | 24.5 MB 11.3 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#s3d feautres
def getDatas3d(id,data,mypath):
    final_dict={}
    step_ids=[]
    step_starts_sec=[]
    step_ends_sec=[]
    duration=data['duration']
    txt_filename=mypath+'/text_embeddings/'+str(id)+'.pt'
    video_filename=mypath+'s3d_features/'+str(id)+'.npy'
    text_feats=torch.load(txt_filename)
    text_features=text_feats["text_embedding"].to(dtype=float)
    video_features=np.load(video_filename)
    video_features=torch.tensor(video_features)
    
    frames=len(video_features)
    
    final_dict['name']=id
    final_dict['cls']=data['recipe_type']
    final_dict['cls_name']=data['recipe_type']
    final_dict['num_frames']=torch.tensor(frames)
    final_dict['frame_features']=video_features
    final_dict['num_steps']=torch.tensor(len(text_features))
    final_dict['step_features']=text_features
    
    for val in data['annotations']:
        step_ids.append(val['id'])
        step_starts_sec.append(val['segment'][0])
        step_ends_sec.append(val['segment'][1])
        
    final_dict['step_ids']=torch.tensor(step_ids)
    final_dict['step_starts_sec']=torch.tensor(step_starts_sec)
    final_dict['step_ends_sec']=torch.tensor(step_ends_sec)
    final_dict['step_ends'] = np.array(
    [int((s*frames)//duration) for s in step_ends_sec]
    )
    final_dict['step_starts'] = np.array(
    [int((s*frames)//duration) for s in step_starts_sec]
    )
    
    return final_dict


In [4]:
# Opening JSON file
f = open('youcookii_annotations_trainval.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
data=data['database']

In [5]:
dataset=[]
mypath='./s3d_features/'
path='./'
i=0
for f in listdir(mypath):
    id=f[:11]
#     print(id)
      
    if id!='PqyqB_FkWrQ' and id in data:
        dataset.append(getDatas3d(id,data[id],path))

In [6]:
print(len(dataset))

1672


In [10]:
model=None
gamma=1
compute_all_metrics(dataset,model,gamma,config)

100%|██████████| 1672/1672 [01:06<00:00, 25.15it/s]


(53.038015380880985,
 31.71463833389377,
 62.95751898223876,
 41.87704176602314,
 11.00881943369952)

In [7]:
class Config:
    def __init__(self, dataset,name,distance,dp_algo,drop_cost,keep_percentile,use_unlabeled):
        self.dataset = dataset
        self.name = name
        self.distance = distance
        self.dp_algo = dp_algo
        self.drop_cost = drop_cost
        self.keep_percentile = keep_percentile
        self.use_unlabeled=use_unlabeled
        

In [8]:
config=Config('YouCook2','','cos','DropDTW','logit',0.3,True)

In [9]:
epochs=10
batch_size=12

In [10]:
from torch.utils.data import DataLoader, sampler
train_dataloader = DataLoader(dataset, batch_size=batch_size,
                                  shuffle=False, num_workers=4)

In [10]:
model = EmbeddingsMapping(
        d=512, learnable_drop=False, video_layers=2,
        text_layers=2, normalization_dataset=None,
        batchnorm=False)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = "cuda" if torch.cuda.is_available() else "cpu"
model=model.to(device)
print(device)

RuntimeError: CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect. For debugging consider passing CUDA_LAUNCH_BLOCKING=1. CUDA error: out of memory

In [9]:
!nvidia-smi

Tue May  3 18:52:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    On   | 00000000:1A:00.0 Off |                    0 |
| 67%   85C    P2   126W / 140W |  15090MiB / 15099MiB |     48%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A4000    On   | 00000000:1B:00.0 Off |                  Off |
| 69%   

In [7]:
from models.nets import EmbeddingsMapping
from models.losses import compute_clust_loss, compute_alignment_loss

In [15]:
from data.batching import unflatten_batch

In [19]:
dataset2=dataset[:10]

In [20]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
batch_dataset=chunks(dataset,24)
len(batch_dataset)

TypeError: object of type 'generator' has no len()

In [22]:
from copy import deepcopy, copy

In [25]:
for ee in range(epochs):
#     total_loss = 0
    for data in dataset2:
        data=data
        total_loss = 0
        sample = copy(data)
#         sample['frame_features'] = pl_module.model.map_video(data['frame_features'].to(device)).detach()
#         sample['step_features'] = pl_module.model.map_text(data['step_features'].to(device)).detach()
#         print(data['frame_features'].float().dtype)
        data['frame_features']=data['frame_features'].float().to(device)
        data['step_features']=data['step_features'].float().to(device)
        sample['frame_features'] = model.map_video(data['frame_features'].to(device))
        sample['step_features'] = model.map_text(data['step_features'].to(device))
        
#         samples=unflatten_batch(sample)
        optimizer.zero_grad()
        
        

        clust_loss = compute_clust_loss([sample], None, xz_hard_ratio=1,
                                      xz_gamma=30, frame_gamma=10,
                                      all_classes_distinct=True,
                                      bg_scope='global')
#         self.log('train/clust_loss', clust_loss)
        total_loss += 4* clust_loss


        dtw_loss = 2.5 * compute_alignment_loss(
            [sample], None, contiguous=True,
            gamma_xz=10, gamma_min=1,
            drop_cost_type='logit', dp_algo='DropDTW',
            keep_percentile=0.3, softning='prob')
#         self.log('train/dtw_loss', dtw_loss)
        total_loss += dtw_loss
        print(total_loss)
        total_loss.backward(retain_graph=True)
        optimizer.step()
        
#         self.log('train/total_loss', self.avg_loss_metric(total_loss))
    print("epoch",ee)
    print("total loss ",total_loss)

function device cuda:1


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [67]:
dataset3=[]
for data in dataset:
    data=data
    total_loss = 0
    sample = copy(data)
#         sample['frame_features'] = pl_module.model.map_video(data['frame_features'].to(device)).detach()
#         sample['step_features'] = pl_module.model.map_text(data['step_features'].to(device)).detach()
#         print(data['frame_features'].float().dtype)
    data['frame_features']=data['frame_features'].float()
    data['step_features']=data['step_features'].float()
    sample['frame_features'] = model.map_video(data['frame_features'])
    sample['step_features'] = model.map_text(data['step_features'])
    dataset3.append(sample)

compute_all_metrics(dataset3,model,gamma,config)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [75]:
!nvidia-smi


Tue May  3 16:18:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    On   | 00000000:B1:00.0 Off |                 Off* |
| 63%   81C    P2   131W / 140W |  16110MiB / 16117MiB |     96%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A4000    On   | 00000000:B2:00.0 Off |                 Off* |
| 60%   